In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [3]:
# Create a Pinecone index
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Psychology',
  'stars': 4,
  'review': 'Dr. Smith is engaging and makes the material easy to understand. However, his exams can be a bit tricky.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Calculus I',
  'stars': 5,
  'review': 'Dr. Davis is fantastic! She explains complex concepts clearly and is very approachable.'},
 {'professor': 'Professor Michael Johnson',
  'subject': 'Modern American History',
  'stars': 3,
  'review': "The subject matter is interesting, but Professor Johnson's lectures can be a bit dry and monotonous."},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Dr. Lee is knowledgeable but her teaching style is very fast-paced. I struggled to keep up with the course.'},
 {'professor': 'Professor Daniel Brown',
  'subject': 'Introduction to Sociology',
  'stars': 4,
  'review': 'Professor Brown is very insightful and encourages class participation. The assign

In [5]:

# This format is compatible with OpenAI's text-embedding-ada-002, but may be different for other embedding models
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-ada-002'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [6]:
processed_data[0]

IndexError: list index out of range

In [ ]:
index = pc.Index("rag")  # In Firebase, this is your collection
index.upsert(
    vectors=processed_data,
    namespace="ns1", # And this would be your documents
)

In [ ]:
index.describe_index_stats()